# Character-Level LSTM

Мы обучим модель на тексте книги с народными сказками, после чего попробуем генерировать новый текст.

**Модель сможет генерировать новый текст на основе текста из народных сказок!**

Общая архитектура RNN:

<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/assets/charseq.jpeg?raw=1" width="500">

In [2]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

## Загрузим данные

Загрузим текстовый файл.

In [3]:
with open('skazki.txt', 'r') as f:
    text = f.read()

Посмотрим первые 100 символов:

In [4]:
text[:200]

'КОТ В САПОГАХ\n\nБыло у мельника три сына, и оставил он им, умирая, всего только мельницу, осла и кота. \n\nБратья поделили между собой отцовское добро без нотариуса и судьи, которые бы живо проглотили вс'

### Токенизация

Выполним преобразование текста в числовое представление.
</br>В ячейках ниже создадим два **словаря** для преобразования символов в целые числа и обратно.

In [5]:
chars = tuple(set(text))
print(chars)

# set(text) собирает уникальные символы (буквы, пробелы, знаки препинания и т. д.) из текста.
# tuple(set(text)) превращает множество в кортеж (упорядоченная структура).

('г', 'ц', 'И', 'з', '?', ' ', '\n', 'ъ', 'Н', 'э', ',', 'п', 'у', 'О', 'е', 'т', 'Р', '2', '4', 'М', '!', 'с', 'я', 'Е', 'а', 'С', 'П', 'Ф', '\xa0', 'Э', 'В', 'к', '–', 'Ь', 'Ч', '(', ';', 'Ш', 'К', ')', '0', 'ф', 'ч', '.', '\t', 'Щ', 'У', 'Ы', '9', 'Ю', 'Ж', 'ж', 'ш', 'Х', 'и', 'Л', 'Т', '1', '8', 'н', 'Б', 'о', 'Й', 'р', 'д', '"', 'ь', 'З', 'Я', 'ы', 'Г', 'ю', 'б', 'Ц', 'л', 'Д', 'м', 'щ', 'А', '*', '/', 'х', '5', 'й', '-', '3', ':', 'в')


In [6]:
int2char = dict(enumerate(chars))
print(int2char)

# enumerate(chars) пронумеровывает каждый символ.
# char2int: сопоставляет символ с числом.

{0: 'г', 1: 'ц', 2: 'И', 3: 'з', 4: '?', 5: ' ', 6: '\n', 7: 'ъ', 8: 'Н', 9: 'э', 10: ',', 11: 'п', 12: 'у', 13: 'О', 14: 'е', 15: 'т', 16: 'Р', 17: '2', 18: '4', 19: 'М', 20: '!', 21: 'с', 22: 'я', 23: 'Е', 24: 'а', 25: 'С', 26: 'П', 27: 'Ф', 28: '\xa0', 29: 'Э', 30: 'В', 31: 'к', 32: '–', 33: 'Ь', 34: 'Ч', 35: '(', 36: ';', 37: 'Ш', 38: 'К', 39: ')', 40: '0', 41: 'ф', 42: 'ч', 43: '.', 44: '\t', 45: 'Щ', 46: 'У', 47: 'Ы', 48: '9', 49: 'Ю', 50: 'Ж', 51: 'ж', 52: 'ш', 53: 'Х', 54: 'и', 55: 'Л', 56: 'Т', 57: '1', 58: '8', 59: 'н', 60: 'Б', 61: 'о', 62: 'Й', 63: 'р', 64: 'д', 65: '"', 66: 'ь', 67: 'З', 68: 'Я', 69: 'ы', 70: 'Г', 71: 'ю', 72: 'б', 73: 'Ц', 74: 'л', 75: 'Д', 76: 'м', 77: 'щ', 78: 'А', 79: '*', 80: '/', 81: 'х', 82: '5', 83: 'й', 84: '-', 85: '3', 86: ':', 87: 'в'}


In [7]:
char2int = {ch: ii for ii, ch in int2char.items()}
print(char2int)

# Обратный словарь: { 'a': 0, 'b': 1, ' ': 2, ... }.

{'г': 0, 'ц': 1, 'И': 2, 'з': 3, '?': 4, ' ': 5, '\n': 6, 'ъ': 7, 'Н': 8, 'э': 9, ',': 10, 'п': 11, 'у': 12, 'О': 13, 'е': 14, 'т': 15, 'Р': 16, '2': 17, '4': 18, 'М': 19, '!': 20, 'с': 21, 'я': 22, 'Е': 23, 'а': 24, 'С': 25, 'П': 26, 'Ф': 27, '\xa0': 28, 'Э': 29, 'В': 30, 'к': 31, '–': 32, 'Ь': 33, 'Ч': 34, '(': 35, ';': 36, 'Ш': 37, 'К': 38, ')': 39, '0': 40, 'ф': 41, 'ч': 42, '.': 43, '\t': 44, 'Щ': 45, 'У': 46, 'Ы': 47, '9': 48, 'Ю': 49, 'Ж': 50, 'ж': 51, 'ш': 52, 'Х': 53, 'и': 54, 'Л': 55, 'Т': 56, '1': 57, '8': 58, 'н': 59, 'Б': 60, 'о': 61, 'Й': 62, 'р': 63, 'д': 64, '"': 65, 'ь': 66, 'З': 67, 'Я': 68, 'ы': 69, 'Г': 70, 'ю': 71, 'б': 72, 'Ц': 73, 'л': 74, 'Д': 75, 'м': 76, 'щ': 77, 'А': 78, '*': 79, '/': 80, 'х': 81, '5': 82, 'й': 83, '-': 84, '3': 85, ':': 86, 'в': 87}


In [8]:
# encode the text
encoded = np.array([char2int[ch] for ch in text])
encoded

# Каждый символ из текста заменяется на его числовое представление, используя словарь char2int.
# Результат сохраняется в массиве encoded (числовая версия текста).

array([38, 13, 56, ..., 43,  6,  6])

Посмотрим как символы закодировались целыми числами

In [9]:
encoded[:100]

array([38, 13, 56,  5, 30,  5, 25, 78, 26, 13, 70, 78, 53,  6,  6, 60, 69,
       74, 61,  5, 12,  5, 76, 14, 74, 66, 59, 54, 31, 24,  5, 15, 63, 54,
        5, 21, 69, 59, 24, 10,  5, 54,  5, 61, 21, 15, 24, 87, 54, 74,  5,
       61, 59,  5, 54, 76, 10,  5, 12, 76, 54, 63, 24, 22, 10,  5, 87, 21,
       14,  0, 61,  5, 15, 61, 74, 66, 31, 61,  5, 76, 14, 74, 66, 59, 54,
        1, 12, 10,  5, 61, 21, 74, 24,  5, 54,  5, 31, 61, 15, 24])

## Предобработка данных

Char-RNN ожидает **one-hot encoded** входа, что означает, что каждый символ преобразуется в целое число (через созданный словарь), а затем преобразуется в вектор-столбец, где только соответствующий ему целочисленный индекс будет иметь значение 1, а остальная часть вектора будет заполнена нулями. Давайте создадим для этого функцию.

In [10]:
def one_hot_encode(arr, n_labels):
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    return one_hot

In [13]:
# check that the function works as expected
test_seq = np.array(encoded[:2])
one_hot = one_hot_encode(test_seq, 87)

print(one_hot)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


## Создаем mini-batch'и


Создадим мини-батчи для обучения. На простом примере они будут выглядеть так:

<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/assets/sequence_batching@1x.png?raw=1" width=500px>
<br>

In [15]:
def get_batches(arr, batch_size, seq_length):
    '''Функция генерирует батчи из массива arr (обычно закодированный текст),
       где каждый батч состоит из входных данных (x) и соответствующих целевых значений (y).
       Она разделяет данные на части фиксированного размера (batch_size x seq_length), подходящие для обучения.

       Arguments
       ---------
       arr: входной массив данных
       batch_size: размер батча: количество последовательностей в одном пакете
       seq_length: длина каждой последовательности внутри батча, количество символов на последовательность
    '''

    batch_size_total = batch_size * seq_length # Общее количество элементов в одном батче
    n_batches = len(arr)//batch_size_total # Вычисляется, сколько полных батчей можно создать из массива. Дробные батчи (остаток) отбрасываются.

    arr = arr[:n_batches * batch_size_total] # Массив обрезается, чтобы его длина делилась нацело на batch_size_total. Это устраняет остаточные данные, которые не вписываются в последний батч.
    arr = arr.reshape((batch_size, -1)) # Массив преобразуется в матрицу размером (batch_size, num_steps_per_batch), где строки представляют отдельные последовательности для каждого батча.

    for n in range(0, arr.shape[1], seq_length): # Проходим по массиву по seq_length, создавая части данных.
        x = arr[:, n:n+seq_length] # Формируется входной батч x, где каждая строка содержит seq_length символов.
        y = np.zeros_like(x) # Создаётся массив y, который будет содержать целевые значения для x.
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length] # Все символы в y (кроме последнего) сдвигаются на один шаг вперёд относительно x. Последний символ берётся из следующего сегмента данных arr[:, n+seq_length].
        except IndexError: # Исключение (IndexError): если выходит за пределы массива (в последнем батче), последний символ связывается с началом массива, замыкая данные в циклический процесс.
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y # Функция возвращает x (входы) и y (цели) как генератор. Генератор позволяет лениво обрабатывать данные, что экономит память.

### Протестируем

Теперь создадим несколько наборов данных, и проверим, что происходит, когда мы создаем батчи.

In [16]:
batches = get_batches(encoded, 8, 50) # Количество строк - 8, Длина батча = 50
x, y = next(batches)

In [27]:
# printing out the first 10 items in a sequence
print('x\n', x[:8, :20])
print('\ny\n', y[:8, :20])

x
 [[38 13 56  5 30  5 25 78 26 13 70 78 53  6  6 60 69 74 61  5]
 [61 74 66 31 12  5 61 59 24  5 72 69 74 24  5 87  5  0 63 22]
 [52 54 15 66 10  5 31 15 61  5 12 64 61 21 15 61 54 15 21 22]
 [43  5 29 15 61 15  5 15 24 59 14  1 10  5 21 61  0 74 24 21]
 [59 14  1  5 61 59 54  5 87 21 11 61 76 59 54 74 54  5 11 63]
 [ 5  6  6 32 28 70 61 15 61 87 24  5 74 54  5 15 69 10  5 21]
 [63 31 87 54  5 59 14  5  3 24  3 87 61 59 22 15  5 31  5  3]
 [ 6 32 28 53 61 63 61 52 61 10  5 84  5 21 31 24  3 24 74  5]]

y
 [[13 56  5 30  5 25 78 26 13 70 78 53  6  6 60 69 74 61  5 12]
 [74 66 31 12  5 61 59 24  5 72 69 74 24  5 87  5  0 63 22  3]
 [54 15 66 10  5 31 15 61  5 12 64 61 21 15 61 54 15 21 22  5]
 [ 5 29 15 61 15  5 15 24 59 14  1 10  5 21 61  0 74 24 21 59]
 [14  1  5 61 59 54  5 87 21 11 61 76 59 54 74 54  5 11 63 61]
 [ 6  6 32 28 70 61 15 61 87 24  5 74 54  5 15 69 10  5 21 14]
 [31 87 54  5 59 14  5  3 24  3 87 61 59 22 15  5 31  5  3 24]
 [32 28 53 61 63 61 52 61 10  5 84  5 21 31 24 

Данные сдвинуты на один шаг для `y`.

## Зададим архитектуру

In [14]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else:
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


<b>Реализуем класс CharRNN</b>, представляющий рекуррентную нейронную сеть (RNN) на основе LSTM (Long Short-Term Memory)
для работы с текстовыми данными.
Она обучается предсказывать следующий символ в последовательности текста.

In [15]:
class CharRNN(nn.Module):

    # Инициализзация параметров модели
    def __init__(self,
                 tokens, # Список уникальных символов текста (например, алфавит), с которыми работает модель.
                 n_hidden=256, # Размер скрытого слоя (количество скрытых нейронов в LSTM). Большее число нейронов позволяет модели выучивать более сложные зависимости.
                 n_layers=2, # Количество слоев LSTM (глубина). Более глубокие модели (с большим количеством слоев) позволяют лучше обрабатывать сложные зависимости в данных.
                 drop_prob=0.5, # Вероятность отключения нейронов (dropout), используется для предотвращения переобучения.
                 lr=0.001 # Скорость обучения. Маленький lr: Сеть обучается медленно. Может найти более точное решение, но есть риск "застрять" в локальном минимуме. Большой lr: Сеть обучается быстро, но есть риск "проскочить" оптимальное решение.
                ):
        
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr

        # Словари преобразуют символы в числа (индексы) и наоборот, чтобы модель могла работать с текстом.
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}

        ## Слой LSTM, обрабатывает последовательности символов и сохраняет скрытое состояние между временными шагами.
        self.lstm = nn.LSTM(
            len(self.chars), # Размер входного слоя равен количеству уникальных символов (вход — закодированный текст).
            n_hidden, # Количество нейронов в скрытом слое.
            n_layers, # Число LSTM-слоев.
            dropout=drop_prob, # Используется между слоями для регуляризации.
            batch_first=True # Формат входных данных (batch_size, seq_length, input_size).
        )

        ## Dropout-слой случайным образом "выключает" нейроны с вероятностью drop_prob, предотвращая переобучение.
        self.dropout = nn.Dropout(drop_prob)

        ## Полносвязный (fully connected) слой преобразует выход LSTM в вероятности для каждого символа. Выходной размер равен количеству уникальных символов (len(self.chars)).
        self.fc = nn.Linear(n_hidden, len(self.chars))

    # Forward-проход (логика прогон данных через модель). x: Входная последовательность символов, закодированная числами. hidden: Текущее скрытое состояние LSTM.
    # Forward-проход задаёт логику всей модели, определяя, как входные данные преобразуются в предсказания.
    def forward(self, x, hidden):
        
        # Пропускаем входные данные через LSTM слой.
        r_output, hidden = self.lstm(x, hidden) # r_output: Результаты LSTM для каждого временного шага.
                                                # hidden: Обновлённое скрытое состояние (используется для следующего временного шага или следующего батча).

        # Пропускаем выход LSTM через Dropout слой.
        out = self.dropout(r_output) # Убирает часть нейронов с вероятностью drop_prob для предотвращения переобучения.

        # Преобразует выходные данные в форму, подходящую для полносвязного слоя.
        # Убирает временные шаги, чтобы слой мог обрабатывать каждый временной шаг как отдельный пример.
        out = out.contiguous().view(-1, self.n_hidden) 

        # Пропускаем данные через полносвязный слой для предсказания символов.
        # Полносвязный слой вычисляет вероятности для каждого символа (на основе скрытого состояния LSTM).
        # Количество выходных нейронов равно количеству символов в словаре (len(self.chars)).
        out = self.fc(out)
            
        # Возвращаем предсказания и обновлённое скрытое состояние.
        return out, hidden # out: вероятности символов. hidden: обновленное состояние для следующего шага.

    ## Инициализация скрытых состояний
    # Функция init_hidden служит для создания начального состояния LSTM перед началом работы с последовательностями.
    def init_hidden(self, batch_size):
        # Создаются тензоры для хранения скрытого состояния (hidden state) и состояния ячейки (cell state) LSTM.
        weight = next(self.parameters()).data

        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())

        return hidden

## Обучим модель

Во время обучения нужно установить количество эпох, скорость обучения и другие параметры.

Используем оптимизатор Adam и кросс-энтропию, считаем loss и, как обычно, выполняем back propagation.

Пара подробностей об обучении:
> * Во время цикла мы отделяем скрытое состояние от его истории; на этот раз установив его равным новой переменной * tuple *, потому что скрытое состояние LSTM, является кортежем скрытых состояний.
* Мы используем [`clip_grad_norm_`](https://pytorch.org/docs/stable/_modules/torch/nn/utils/clip_grad.html) чтобы избавиться от взрывающихся градиентов.

Эта функция train реализует процесс обучения LSTM-модели.
</br>Она содержит шаги, необходимые для тренировки рекуррентной нейронной сети с использованием данных, оценки ошибки и валидации.

In [29]:
# Инициализзация параметров
def train(
    net, # модель, которую нужно обучить.
    data, # данные для обучения (последовательности символов или индексов).
    epochs=10, # количество эпох (полных проходов через данные).
    batch_size=10, # количество последовательностей, обрабатываемых за один шаг.
    seq_length=50, # длина каждой последовательности (число временных шагов).
    lr=0.001, # скорость обучения (learning rate).
    clip=5, # значение для ограничения градиентов, чтобы избежать их взрыва.
    val_frac=0.1, # доля данных, выделяемых для валидации.
    print_every=10 # частота вывода метрик обучения (каждые print_every шагов).
):
  
    net.train() # Установка модели в режим обучения. Это сообщает PyTorch, что модель будет обучаться (активируются такие механизмы, как Dropout).

    # Оптимизатор и функция потерь
    opt = torch.optim.Adam(net.parameters(), lr=lr) # алгоритм оптимизации, который адаптирует скорость обучения для каждого параметра модели.
    criterion = nn.CrossEntropyLoss() # измеряет ошибку между предсказаниями модели и истинными значениями, что помогает модели улучшать свои предсказания в задаче классификации.

    # Разделение данных на обучающие и валидационные
    val_idx = int(len(data)*(1-val_frac)) # val_frac определяет долю данных для валидации (по умолчанию 10%).
    data, val_data = data[:val_idx], data[val_idx:] # Данные делятся на обучающие и валидационные.

    if(train_on_gpu):
        net.cuda()

    counter = 0
    n_chars = len(net.chars)
    
    # Основной цикл обучения. За каждую эпоху модель проходит через все данные.
    for e in range(epochs):
        h = net.init_hidden(batch_size) # Скрытое состояние и состояние ячейки инициализируются для каждого батча.

        # Данные разбиваются на батчи функцией get_batches. В каждом батче:
        # x: входные последовательности.
        # y: целевые значения (следующие символы для предсказания).
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1

            # Входные данные кодируются в формате one-hot (матрица, где каждая строка соответствует символу).
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)

            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Создание новых переменных для скрытого состояния
            h = tuple([each.data for each in h])

            # Сброс градиентов и прогон данных через модель
            # Градиенты обнуляются перед обратным распространением.
            # Вычисляется выход модели и обновляется скрытое состояние.
            net.zero_grad()
            output, h = net(inputs, h)

            # Вычисление потерь и обратное распространение ошибки
            # Функция потерь вычисляет ошибку между предсказаниями модели и целевыми значениями.
            loss = criterion(output, targets.view(batch_size*seq_length).long()) # targets.view(batch_size*seq_length).long() приводит целевые значения к нужной форме.
            loss.backward()
            
            # Ограничение градиентов. Для предотвращения "взрыва градиентов" их величины ограничиваются значением clip.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            
            opt.step() # Обновление параметров модели

            # Валидация на части данных
            if counter % print_every == 0:
                # Получаем validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                
                # Каждые print_every шагов вычисляются метрики на валидационных данных
                # Это помогает следить за качеством модели и избегать переобучения.
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)

                    # Создание новых переменных для скрытого состояния
                    val_h = tuple([each.data for each in val_h])

                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())

                    val_losses.append(val_loss.item())

                net.train() # сброс в режим обучения после итерации validation data

                # Вывод метрик обучения
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

## Определим модель

Теперь мы можем создать модель с заданными гиперпараметрами. Определим размеры мини-батчей.

In [17]:
# define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(88, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=88, bias=True)
)


### Установим гиперпараметры

In [25]:
batch_size = 128
seq_length = 100
n_epochs = 20 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/20... Step: 10... Loss: 1.8545... Val Loss: 1.7506
Epoch: 1/20... Step: 20... Loss: 1.8384... Val Loss: 1.7345
Epoch: 1/20... Step: 30... Loss: 1.8323... Val Loss: 1.7305
Epoch: 1/20... Step: 40... Loss: 1.8114... Val Loss: 1.7237
Epoch: 1/20... Step: 50... Loss: 1.8359... Val Loss: 1.7188
Epoch: 1/20... Step: 60... Loss: 1.8609... Val Loss: 1.7139
Epoch: 1/20... Step: 70... Loss: 1.8314... Val Loss: 1.7106
Epoch: 1/20... Step: 80... Loss: 1.8186... Val Loss: 1.7066
Epoch: 1/20... Step: 90... Loss: 1.8446... Val Loss: 1.7057
Epoch: 1/20... Step: 100... Loss: 1.8349... Val Loss: 1.6995
Epoch: 2/20... Step: 110... Loss: 1.7720... Val Loss: 1.6968
Epoch: 2/20... Step: 120... Loss: 1.8034... Val Loss: 1.6939
Epoch: 2/20... Step: 130... Loss: 1.7563... Val Loss: 1.6913
Epoch: 2/20... Step: 140... Loss: 1.7577... Val Loss: 1.6872
Epoch: 2/20... Step: 150... Loss: 1.7758... Val Loss: 1.6855
Epoch: 2/20... Step: 160... Loss: 1.7668... Val Loss: 1.6780
Epoch: 2/20... Step: 170... Loss:

Epoch: 14/20... Step: 1350... Loss: 1.4888... Val Loss: 1.4621
Epoch: 14/20... Step: 1360... Loss: 1.4932... Val Loss: 1.4620
Epoch: 14/20... Step: 1370... Loss: 1.4719... Val Loss: 1.4593
Epoch: 14/20... Step: 1380... Loss: 1.4758... Val Loss: 1.4616
Epoch: 14/20... Step: 1390... Loss: 1.4531... Val Loss: 1.4579
Epoch: 14/20... Step: 1400... Loss: 1.4904... Val Loss: 1.4583
Epoch: 14/20... Step: 1410... Loss: 1.4819... Val Loss: 1.4601
Epoch: 15/20... Step: 1420... Loss: 1.4855... Val Loss: 1.4566
Epoch: 15/20... Step: 1430... Loss: 1.5010... Val Loss: 1.4555
Epoch: 15/20... Step: 1440... Loss: 1.4677... Val Loss: 1.4544
Epoch: 15/20... Step: 1450... Loss: 1.4604... Val Loss: 1.4540
Epoch: 15/20... Step: 1460... Loss: 1.4502... Val Loss: 1.4565
Epoch: 15/20... Step: 1470... Loss: 1.4539... Val Loss: 1.4537
Epoch: 15/20... Step: 1480... Loss: 1.5003... Val Loss: 1.4577
Epoch: 15/20... Step: 1490... Loss: 1.4757... Val Loss: 1.4514
Epoch: 15/20... Step: 1500... Loss: 1.4966... Val Loss:

## Checkpoint

После обучения сохраним модель, чтобы можно было загрузить ее позже. Здесь сохраняются параметры, необходимые для создания той же архитектуры, гиперпараметры скрытого слоя и токены.

In [26]:
# change the name, for saving multiple files
model_name = 'rnn_x_epoch_ru.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

---
## Делаем предсказания

Теперь, когда мы обучили модель, сделаем предсказание следующих символов! Для предсказания мы передаем последний символ, и сеть предсказывает следующий символ, который мы потом передаем снова на вхол и получаем еще один предсказанный символ и так далее...

Наши прогнозы основаны на категориальном распределении вероятностей по всем возможным символам. Мы можем ограничить число символов, чтобы сделать получаемый предсказанный текст более разумным, рассматривая только некоторые наиболее вероятные символы $K$. Это не позволит сети выдавать нам совершенно абсурдные прогнозы, а также позволит внести некоторый шум и случайность в выбранный текст. Узнать больше [можно здесь](https://pytorch.org/docs/stable/generated/torch.topk.html#torch.topk).

In [27]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''

        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)

        if(train_on_gpu):
            inputs = inputs.cuda()

        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu

        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()

        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())

        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

### Priming и генерирование текста

Нужно задать скрытое состояние, чтобы сеть не генерировала произвольные символы.

In [28]:
def sample(net, size, prime='Король', top_k=None):

    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()

    net.eval() # eval mode

    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)

    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [29]:
print(sample(net, 1000, prime='Золото', top_k=5))

Золотой. 

И король осла в сад положил к стола и плачу, что не положили его на королевской купочке, платье и веснились и стали принца сверкать на белых соной, поклинывались на пострую своего прочь, и в одной комнате подумала стоят крыльями. Прошел королевна и воскилась на плика. 

– Как же мой слова перескажите, когда он не поведет на собой принцессе! - возразили платья. - Но откуда то смерть спасеби тебе серебряное столковые слугу. 

Вот как только они очно сказала: 

– Не знаю что! Пока не возьму на свете! 

Принц отпустился в облакой колонец, полно ответил: 

– Нет, не было слезом, поже нам проводить свою другой колени. 

И тот королевич сказал: 

– Как бы твое нужное было послышно подоши в станику? Но я стала на ней самому стольную дверь все, что приданое войско. Ты посмотрел на него не помогу, но нам наколеть проснулся! 

– Ну поставил мне следуют королевскай мышь и волшебницы, - ответил середин и поставил на королевский дворец. 

На другой день она сказала: 

– Принесли, тебя был

## Loading a checkpoint

In [31]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_x_epoch_ru.net', 'rb') as f:
    checkpoint = torch.load(f)

loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [32]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="Золото"))

Золотой Золотой короля во сне в старой комнате на две невидиные два самого совета. 

Но сказал ему, что она в семь дом поставила в прослый колдуньи, а прявлятся несколько мир. Они все белых детем с какому не привели несколько дней и в сердце ведел сважить. 

– Не скажи то, что все это не бежать, привысти мне служить перед ними солатье стороного... 

– Пусть она стала тебе, - сказала она. - Так он покупайте твоей жизни! 

Перед ним на дво слова подумала с ним по сторонам, когда он пришлась спасить на колени. 

Поднял сестра, показали все в камни и обрадалась превратиться. Пока отец ного в стороне он увидел на своих статьях, и она стала насметно вороном слуги и подняла королевну. 

– Когда же мне быть соберальный дворец и положился? - ответил колокольки, покрытые стол. 

– Не подуму все, что страшно быть нет навеста. 

– А как же мы не пойдем в моей жене? 

– Пришлось меня, которой спраслало ему, чтобы наста надежда с ней называться. Надо того он начили принц. 

Великан взял великан с ко